<a href="https://colab.research.google.com/github/poojitha-lankipalle/PBDMS_Final/blob/master/Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import auth
import gspread
from google.auth import default
from IPython.display import clear_output
import time
import pandas as pd

# Authenticate and authorize
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# Open the Google Sheets document and select the worksheet
wb = gc.open_by_key('194pjbJBEL1pO55lPQj_yCdjV4JHbFiQohFHqR8UJem4')
ws = wb.worksheet('Sheet1')

while True:

    data = ws.get_all_values()


    df = pd.DataFrame(data[1:], columns=data[0])


    clear_output(wait=True)
    display(df)


    time.sleep(10)



,Name,Age,BloodPressure,BMI,Glucose,totChol,Location,BP Alert,Sugar Alert,Phone numbers
0,Mary,50,77,33.6,193,195,"88.650386, 85.215967",,Blood pressure is Increasing,9133787157
1,Anna,31,138,26.6,232,250,"-13.732284, -87.520661",Blood pressure is Incresing,Blood pressure is Increasing,
2,Emma,32,164,23.3,180,245,"-63.752196, 45.089570",Bllod pressure is Decreasing,,9133787157
3,Elizabeth,21,122,28.1,179,225,"83.581105, -92.181284",,Sugar levels are high,9133787157
4,Minnie,33,89,43.1,71,285,"60.371180, -107.293943",,Sugar levels are high,
...,...,...,...,...,...,...,...,...,...,...
496,Pauline,30,151,26,94,245,"18.819955, -54.863709",,,
497,Rose,25,97,30.1,122,233,"-38.566640, -128.001227",,,
498,Sally,55,83,25.1,87,258,"82.116599, -110.583191",,,
499,nancy,39,135,29.3,160,215,"72.310698, -176.298836",,,


KeyboardInterrupt: ignored

# New Section

In [ ]:
import time
from twilio.rest import Client

# Your Twilio account SID and auth token
twilio_account_sid = 'AC2ca6daaba6958f84f859db12b219b43f'
twilio_auth_token = '8d5a90192eeae1a74c072b49e391b83b'
twilio_phone_number = '+18775221937'

row_limit = 50  # Specify the row index to end the code

hospital_data = {
    'Hospital': {'latitude': 40.7128, 'longitude': -74.0060},

}

# Create a Twilio client
twilio_client = Client(twilio_account_sid, twilio_auth_token)

while True:
    # Get all values from the first column to check
    blood_pressure = ws.col_values(3)

    # Get all values from the second column to check
    Sugar = ws.col_values(5)

    # Get all values from the G column for additional data in the message body
    Location = ws.col_values(7)

    Name = ws.col_values(1)

    # Get all values from the J column for phone numbers
    phone_numbers = ws.col_values(10)

    # Loop through each row and check the condition for both columns simultaneously
    for i, (value1, value2, location, phone_number,name) in enumerate(zip(blood_pressure[1:], Sugar[1:], Location[1:], phone_numbers[1:],Name[1:]), start=2):  # Skip the first row (header) and start from the second row (row index 2)
        if i > row_limit:
            break  # Break the loop when the row index exceeds the specified limit

        if phone_number:  # Check if the phone number is not empty

            formatted_phone_number = "+1" + phone_number

            hospital_name = 'Hospital'
            hospital_coordinates = hospital_data[hospital_name]
            hospital_latitude = hospital_coordinates['latitude']
            hospital_longitude = hospital_coordinates['longitude']


            body = f'You are listed as an Emergency contact for {name}. Blood pressure is : {value1}, Sugar levels are: {value2}, Location details of {name} are : {location}.Hospital {hospital_name} coordinates are: Latitude: {hospital_latitude}, Longitude: {hospital_longitude}. Please get in touch'

            if float(value1) > 120:
                # Update the corresponding columns for the same row with the desired values
                ws.update_cell(i, 8, 'Blood pressure is Increasing')
                twilio_client.messages.create(
                    body=body,
                    from_=twilio_phone_number,
                    to=formatted_phone_number
                )
            elif float(value1) < 70:
                ws.update_cell(i, 8, 'Blood pressure is Decreasing')
                twilio_client.messages.create(
                    body=body,
                    from_=twilio_phone_number,
                    to=formatted_phone_number
                )
            else:
                ws.update_cell(i, 8, '')

            if float(value2) > 170:
                ws.update_cell(i, 9, 'Sugar levels are high')
                twilio_client.messages.create(
                    body=body,
                    from_=twilio_phone_number,
                    to=formatted_phone_number
                )
            elif float(value2) < 70:
                ws.update_cell(i, 9, 'Sugar levels are low')
                twilio_client.messages.create(
                    body=body,
                    from_=twilio_phone_number,
                    to=formatted_phone_number
                )
            else:
                ws.update_cell(i, 9, '')

    time.sleep(10)


KeyboardInterrupt: ignored

In [ ]:
!pip install twilio



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 20.3 MB/s eta 0:00:00
